In [111]:
import mcpi.minecraft as minecraft
import mcpi.block as block
import mcpi.entity as entity
from mcpi.vec3 import Vec3
import numpy as np
import random
import pickle
import glob
import time
import os

In [11]:
mc = minecraft.Minecraft.create(address='192.168.1.100')

In [117]:
side = 9
p = mc.player.getPos()
p.y = p.y-1
for x in np.arange(0, side, 1):
    for y in np.arange(0, side, 1):
        for z in np.arange(0, side, 1):
            if x==0 or x==side-1 or y==0 or y==side-1 or z==0 or z==side-1:
                mc.setBlock(p.x+x, p.y+y, p.z+z, block.WOOL_LIGHT_BLUE)
                


In [118]:
print(p)
coords = {}
coords['bathroom'] = Vec3(171,0,158)
coords['dining'] = Vec3(319,-1,137)
coords['library'] = Vec3(336, -1, 96)
coords['factory'] = Vec3(366, -1, 75)
coords['clothes_shop'] = Vec3(399, -1, 48)
coords['swimming_pool'] = Vec3(377, -1, -32)

Vec3(377.98952209469326,-1.0,-32.95217852168787)


In [116]:
for r in coords.keys():
    print('Processing %s...' % r)
    blocks = mc.getBlocksWithNBT(coords[r].x, coords[r].y, coords[r].z, coords[r].x+side-1, coords[r].y+side-1, coords[r].z+side-1)
    room_spec = {
        'name': r,
        'side': 9,
        'door_style': 'middle',
        'source_location': coords[r],
        'version': int(time.time()),
        'blocks': blocks,
    }
    with open('rooms/%s.pickle' % r, 'wb') as f:
        pickle.dump(room_spec, f)



Processing bathroom...
Processing dining...
Processing library...
Processing factory...
Processing clothes_shop...


In [124]:
rooms = {}
for rf in glob.glob("rooms/*.pickle"):
    with open(rf,'rb') as f:
        room = pickle.load(f)
        rooms[room['name']] = room
print(rooms['bathroom'])


{'name': 'bathroom', 'side': 9, 'door_style': 'middle', 'source_location': Vec3(171,0,158), 'version': 1661037099, 'blocks': [Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(57, 0), Block(

In [121]:
cp = mc.player.getPos()
mc.player.setPos(-1, 0, -1)


In [104]:
mc.player.setPos(cp)

In [128]:
i = 0
for y in np.arange(0, side, 1):
    for x in np.arange(0, side, 1):
        for z in np.arange(0, side, 1):
            mc.setBlockWithNBT(0.0+x, 0.0+y, 0.0+z, rooms['factory']['blocks'][i])
            i += 1

In [42]:
i = 0
for b in bathroom:
    print(i, b)
    i += 1



0 Block(57, 0)
1 Block(57, 0)
2 Block(57, 0)
3 Block(57, 0)
4 Block(57, 0)
5 Block(57, 0)
6 Block(57, 0)
7 Block(57, 0)
8 Block(57, 0)
9 Block(0, 0)
10 Block(57, 0)
11 Block(57, 0)
12 Block(57, 0)
13 Block(57, 0)
14 Block(57, 0)
15 Block(57, 0)
16 Block(57, 0)
17 Block(57, 0)
18 Block(57, 0)
19 Block(0, 0)
20 Block(57, 0)
21 Block(57, 0)
22 Block(57, 0)
23 Block(57, 0)
24 Block(57, 0)
25 Block(57, 0)
26 Block(57, 0)
27 Block(57, 0)
28 Block(57, 0)
29 Block(0, 0)
30 Block(57, 0)
31 Block(57, 0)
32 Block(57, 0)
33 Block(57, 0)
34 Block(57, 0)
35 Block(57, 0)
36 Block(57, 0)
37 Block(57, 0)
38 Block(57, 0)
39 Block(0, 0)
40 Block(57, 0)
41 Block(57, 0)
42 Block(57, 0)
43 Block(57, 0)
44 Block(57, 0)
45 Block(57, 0)
46 Block(57, 0)
47 Block(57, 0)
48 Block(57, 0)
49 Block(0, 0)
50 Block(57, 0)
51 Block(57, 0)
52 Block(57, 0)
53 Block(57, 0)
54 Block(57, 0)
55 Block(57, 0)
56 Block(57, 0)
57 Block(57, 0)
58 Block(57, 0)
59 Block(0, 0)
60 Block(57, 0)
61 Block(57, 0)
62 Block(57, 0)
63 Block

In [82]:
i = 0
for y in np.arange(-1, 256, 1):
    for x in np.arange(0, side, 1):
        for z in np.arange(0, side, 1):
            mc.setBlockWithNBT(0.0+x, 0.0+y, 0.0+z, block.AIR)
            i += 1